# Exploratory Data Analysis (EDA)

Bu notebook, proje klasörünüzdeki **train_features.csv** ve **test_features.csv** dosyalarını okuyup;
1. Librosa ile çıkarılan **özellikleri** (MFCC, spectral centroid vb.) inceler,
2. Bazı **istatistiksel dağılımları** ve **görselleri** sunar,
3. Klasik **Makine Öğrenmesi** yöntemlerinin (XGBoost, Naïve Bayes, Random Forest, KNN, Logistic Regression, SVM, MLP) doğruluklarını karşılaştırır,
4. Bazı **Derin Öğrenme** tabanlı (CNN, RNN, LSTM, EfficientNet, ResNet, DenseNet, MobileNet) yöntemlerin örnek sonuçlarını kıyaslar,
5. Son olarak **karşılaştırma** ve **genel yorum** yapar.

**Not:** Bu notebook içinde model eğitim kodlarını *tamamen* tekrar etmiyoruz; çoğu sonuç "örnek skorlar" şeklinde gösterilecektir. Projenizdeki eğitim betiklerini (train.py vb.) ayrı tutmak daha düzenli olur.


## Gerekli Kütüphaneler

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Grafiklerin notebook içinde gösterilmesi
%matplotlib inline


ModuleNotFoundError: No module named 'seaborn'

## 1) Veri Yükleme

Proje klasörünüzdeki **train_features.csv** ve **test_features.csv** dosyalarını okuyalım. Bu dosyalar, feature_extraction.py ile elde ettiğiniz sütunları içeriyor olmalı.

In [2]:
# Veri yükleme
train_df = pd.read_csv('train_features.csv')  # proje klasörünüzdeki dosya
test_df = pd.read_csv('test_features.csv')    # proje klasörünüzdeki dosya

print('Train set shape:', train_df.shape)
print('Test set shape :', test_df.shape)

train_df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'train_features.csv'

Gözlemlediğimiz gibi, her satır **bir ses dosyasına** ait çıkarılmış özellikleri temsil ediyor. **file_path** ve **label** sütunları haricindeki bütün numerik sütunlar (ör. spectral_centroid_mean, mfcc_1_skew vb.) model girdi (feature) olarak kullanılabilir.

### Sütun İsimleri ve Sayısı

In [ ]:
all_columns = train_df.columns.tolist()
print('Tüm sütunlar:', all_columns)
print('\nSütun sayısı:', len(all_columns))

Proje kapsamında Librosa ile çıkarmış olduğumuz **özellik sütunları** (ör. spectral_centroid, spectral_flux, tempo, MFCC'ler vb.) bu listede yer alıyor. 

## 2) Bazı Özelliklerin İstatistiksel İncelenmesi
Hızlıca bir describe() ve temel grafiklerle dağılımlara göz atalım.

In [ ]:
# Numeric sütunları ayıklıyoruz (file_path ve label'ı çıkararak)
feature_cols = [c for c in train_df.columns if c not in ['file_path', 'label']]

train_df[feature_cols].describe().T.head(15)

İsterseniz seçtiğiniz birkaç özelliğin histogramını çizdirerek dağılımlara bakabiliriz. Örneğin spectral_centroid_mean, spectral_flux_mean, mfcc_1_mean gibi sütunlar.

In [ ]:
cols_to_plot = [
    'spectral_centroid_mean', 
    'spectral_flux_mean', 
    'mfcc_1_mean'
]

plt.figure(figsize=(12,4))
for i, col in enumerate(cols_to_plot, 1):
    plt.subplot(1, len(cols_to_plot), i)
    sns.histplot(train_df[col], kde=True, color='teal')
    plt.title(col)
plt.tight_layout()
plt.show()

### Korelasyon Matrisi (Örnek)
Fazla özelliğiniz varsa tümüyle korelasyon çizmek kalabalık olur, o yüzden sadece birkaçını seçelim.

In [ ]:
# Küçük bir alt set
subset_cols = ['spectral_centroid_mean', 'spectral_flux_mean', 'spectral_bandwidth_mean', 'mfcc_1_mean']
corr = train_df[subset_cols].corr()

plt.figure(figsize=(5,4))
sns.heatmap(corr, annot=True, cmap='YlGnBu', center=0)
plt.title('Küçük bir korelasyon matrisi')
plt.show()

## 3) Klasik Makine Öğrenmesi Modellerinin Karşılaştırılması
Projenizde, bu özellikleri kullanarak XGBoost, Naïve Bayes (NB), Random Forest, K-En Yakın Komşu (KNN), Lojistik Regresyon (LR), Destek Vektör Makineleri (SVM) ve bir Derin Sinir Ağı (MLP) eğitmiş olabilirsiniz.

Aşağıda, bu **7** modelin doğruluk skorlarını örnek tablo ve grafikle gösteriyoruz. Gerçek projede bu skorları, kendi train.py vb. kodunuzdan alıp bu notebook'a ekleyebilirsiniz.

In [ ]:
# Örnek (mock) sonuçlar:
ml_results = {
    'Model': [
        'XGBoost', 'Naive Bayes', 'Random Forest', 'KNN', 'Logistic Regression', 'SVM', 'MLP (DNN)'
    ],
    'Accuracy (%)': [
        88.5, 75.3, 85.1, 78.6, 80.2, 83.7, 86.4  # Örnek değerler
    ]
}

df_ml = pd.DataFrame(ml_results)
df_ml

### Barplot ile Karşılaştırma

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=df_ml, x='Model', y='Accuracy (%)', palette='viridis')
plt.ylim(0,100)
plt.title('Klasik ML Modelleri - Doğruluk Karşılaştırması')
plt.xticks(rotation=45)
plt.show()

Genelde **XGBoost** ve **Random Forest** gibi ağaç tabanlı yöntemlerin yüksek performans verdiği görülüyor. SVM de veriye göre iyi sonuçlar verebilir. NB ve KNN ise veri setinin boyut, dağılım ve özelliklerine göre orta düzeyde kalabilir.


## 4) Derin Öğrenme Tabanlı Modellerin Karşılaştırılması
Proje kapsamında, **CNN, RNN, LSTM, EfficientNet, ResNet, DenseNet, MobileNet** gibi modelleri ses verisine uygulayan senaryolara da rastlanabilir. Örneğin, spectrogram görüntüleri üzerinden CNN tabanlı bir model.

Aşağıdaki tablo ve grafik yine "örnek sonuçlar" içermektedir.

In [ ]:
deep_results = {
    'Model': ['CNN', 'RNN', 'LSTM', 'EfficientNet', 'ResNet', 'DenseNet', 'MobileNet'],
    'Accuracy (%)': [
        89.2, 84.6, 86.7, 91.3, 90.1, 88.9, 89.5  # Örnek skorlar
    ]
}

df_deep = pd.DataFrame(deep_results)
df_deep

### Barplot ile Derin Öğrenme Modelleri Karşılaştırması

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=df_deep, x='Model', y='Accuracy (%)', palette='magma')
plt.ylim(0,100)
plt.title('Derin Öğrenme Modelleri - Doğruluk Karşılaştırması')
plt.xticks(rotation=45)
plt.show()

Burada **EfficientNet** ve **ResNet** biraz daha yüksek doğrulukla öne çıkıyor gibi görünüyor. Tabii gerçek verilerde hangi modelin iyi olacağı, veri boyutu ve hyperparameter tuning'e göre değişebilir.


## 5) Sonuç ve Değerlendirme

- **Librosa ile** çıkardığımız ~170+ özelliğin dağılımlarını inceledik; MFCC, spectral vb. her bir özelliğe 6 istatistik (mean, std, min, max, kurtosis, skew) eklenince geniş bir tablo oluşuyor.
- **Klasik ML** yöntemlerinde, XGBoost ve Random Forest genellikle iyi performans gösteriyor. SVM de iddialı olabilir. NB ve KNN genelde daha düşük sonuçlar alabiliyor.
- **Derin Öğrenme** yaklaşımında CNN, RNN, LSTM gibi temel mimariler yanında, EfficientNet veya ResNet gibi modellerin de ses spectrogramlarına uygulanmasıyla yüksek doğruluk elde etmek mümkün.

### Öneriler
1. **Data Augmentation** (pitch shift, time stretch, noise injection vb.) yaparak veri çeşitliliğini artırın.
2. Model hyperparametrelerini (learning rate, batch size, dropout, vb.) daha detaylı arayarak iyileştirin.
3. Farklı metriklerle (f1-score, recall, precision) de değerlendirme yapıp dengesiz veri durumlarına bakın.
4. Ses verisi doğal ve uzun kayıtlar ise **chunk bazlı** veya **VAD (Voice Activity Detection)** kullanarak daha doğru analiz edebilirsiniz.

Bu çalışma, temel bir EDA ve model sonuçları karşılaştırması sağlamıştır. Projenizin boyutu ve hedef duygu sınıflarının genişliği arttıkça, veri toplama ve model tasarımı da önem kazanacaktır.